## Using ML anonymization to defend against attribute inference attacks

### Load data

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

import warnings
# Filter out all warnings
warnings.filterwarnings("ignore")


#### First of all, we need to import the required packages to perform our privacy analysis and mitigation. You will need to have the `holisticai` package installed on your system, remember that you can install it by running: 
```bash
!pip install holisticai[all]
```

In [2]:
from holisticai.datasets import load_dataset

dataset = load_dataset('adult', preprocessed=False)
dataset

Dataset({
        features: ['X', 'y', 's'],
        num_rows: 45222
    })

In [3]:
X = dataset["X"].drop('fnlwgt', axis=1)
from holisticai.datasets import Dataset
new_dataset = Dataset(X=X, y=dataset["y"], s=dataset["s"])
new_dataset

Dataset({
        features: ['X', 'y', 's'],
        num_rows: 45222
    })

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


train_test = new_dataset.train_test_split(0.2, random_state=42)
train = train_test['train']
test = train_test['test']

# Identify categorical and numerical features
def create_preprocessor(X):
    categorical_features = X.select_dtypes(include=['category']).columns
    numerical_fatures = X.select_dtypes(exclude=['category']).columns

    # Create transformers for numerical and categorical features
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

    # Combine transformers into a preprocessor using ColumnTransformer
    return ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_fatures),
            ('cat', categorical_transformer, categorical_features)
    ])

### Train decision tree model

In [5]:
from sklearn.tree import DecisionTreeClassifier

preprocessor = create_preprocessor(train['X'])
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', DecisionTreeClassifier())])

model.fit(train['X'], train['y'])

y_pred = model.predict(test['X'])
y_proba = model.predict_proba(test['X'])

print('Base model accuracy: ', model.score(test['X'], test['y']))

Base model accuracy:  0.8210060807075732


### BlackBox Attack

In [ ]:
from holisticai.security.commons import classification_security_features
from holisticai.security.metrics import attribute_inference_accuracy

attacker = classification_security_features(X=train['X'], y=train['y'], attacker='black_box', attack_feature='education')
attribute_inference_accuracy(attacker, test['X'], test['y'])

#### This means that for 89% of the training set, the attacked feature is inferred correctly using this attack.



### Anonymized data. Improving privacy 

In [7]:
from holisticai.security.mitigation import Anonymize

X_train = train['X']
y_train = train['y']

feature_names = X_train.columns
categorical_features = list(X_train.select_dtypes(include=['category']).columns)
QI = ['education', 'marital-status', 'age']

anonymizer = Anonymize(500, QI, categorical_features=categorical_features, features_names=feature_names)
anon = anonymizer.anonymize(X_train, y_train)

### Train decision tree model on anonymized data


In [ ]:
from sklearn.tree import DecisionTreeClassifier

preprocessor = create_preprocessor(train['X'])
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', DecisionTreeClassifier())])

model.fit(anon, train['y'])

y_pred = model.predict(test['X'])
y_proba = model.predict_proba(test['X'])

print('Base model accuracy: ', model.score(test['X'], test['y']))

In [ ]:
from holisticai.security.commons import classification_security_features
from holisticai.security.metrics import attribute_inference_accuracy

attacker = classification_security_features(X=anon, y=train['y'], attacker='black_box', attack_feature='education')
attribute_inference_accuracy(attacker, test['X'], test['y'])

#### This means that for 56% of the training set, the attacked feature is inferred correctly using this attack.
